In [ ]:
#| default_exp utils.asian_1x2_pnl

In [ ]:
#| hide

from IPython.core.debugger import set_trace

%load_ext autoreload
%autoreload 2

# 1X2, Asian Handicap and Total Goal Profit
> Profit for 1X2, Asian handicap and total goals betting

In [ ]:
#| export

import mongoengine
import numpy as np
import pandas as pd

Using the `config.toml` credentials included in the main repository, we must first load games data stored in our MongoDb Cluster before testing our betting environment.

In [ ]:
from fastbet.config.localconfig import CONFIG, DB_HOSTS
from fastbet.config.mongo import mongo_init
from fastbet.datastructure.data_extractor import *

In [ ]:
mongo_init(db_host="public_atlas")

fixture = data_aggregator(limit=10)
fixture

  0%|          | 0/10 [00:00<?, ?it/s]

,gameId,game_optaId,gameDate,homeTeamId,homeTeam_optaId,awayTeamId,awayTeam_optaId,tgt_gd,tgt_outcome,tgt_homeTeamGoals,...,homeTeamLineupIds,homeTeamLineupSlots,homeTeamFormation,home_team_lineup_received_at,awayTeamName,awayTeamLineup,awayTeamLineupIds,awayTeamLineupSlots,awayTeamFormation,away_team_lineup_received_at
0,c0c48eee0b1a42e0d84cb0a947fe2c64f9e1aa7015922f...,990998,2018-08-21 18:45:00,bc9d5de208258f2f95282c59e9551310be9d319ebc6e4e...,24,4a625f945d8f58984be0aa7b2ac6409a23ed9cf48e4260...,40,2,0.0,2,...,"[106423, 12744, 184341, 57714, 103920, 17601, ...","[4, 9, 8, 3, 10, 1, 2, 5, 6, 11, 7]",4-3-3,2018-08-21 18:15:00,Ipswich Town,"{""Jonas Knudsen"": ""DL"", ""Janoi Donacien"": ""DR""...","[82187, 154936, 101881, 115557, 28530, 19910, ...","[3, 2, 10, 7, 1, 8, 9, 11, 4, 5, 6]",4-1-4-1,2018-08-21 18:15:00
1,3a604f5616b39eb17fc8d1eed07d5248e387bf400294b2...,991000,2018-08-21 18:45:00,e2bfbb5453a7853e049b9434db74d4d06b8c5560ff7cf9...,52,d6fe4a4ffbf1e1a0ae9d4bbed16e94042d9bf01e57eb55...,113,-3,2.0,0,...,"[91068, 89184, 49083, 106606, 42996, 95767, 23...","[10, 1, 8, 7, 2, 4, 11, 3, 6, 5, 9]",4-3-3,2018-08-21 18:15:00,Bristol City,"{""Josh Brownhill"": ""DMR"", ""Jack Hunt"": ""DR"", ""...","[172782, 73716, 55563, 110735, 106257, 235530,...","[4, 2, 7, 6, 10, 3, 8, 5, 11, 1, 9]",4-4-2,2018-08-21 18:15:00
2,58b1242154c8055252582229abfc4680460278834c4433...,991001,2018-08-21 18:45:00,58301066042bbdf19de8fe7d41afc53626b5aa79034712...,72,bbb63e4ea54b0d60b48a1f8440254d7e656dfbfcbef825...,88,-1,2.0,2,...,"[80246, 19152, 193576, 155529, 124120, 41753, ...","[7, 6, 2, 1, 4, 3, 11, 8, 5, 9, 10]",4-5-1,2018-08-21 18:15:00,Hull City,"{""Evandro Goebel"": ""AMC"", ""Jordy De Wijs"": ""DC...","[52287, 173549, 120449, 28541, 107692, 178186,...","[10, 6, 2, 9, 11, 7, 4, 5, 3, 1, 8]",4-4-1-1,2018-08-21 18:15:00
3,d0cc49c3230e300b529b270951b3b70b3224481add8354...,991007,2018-08-21 18:45:00,9ca1f9a87934693b07890de4b4528b0f3ae4065a67ec38...,80,38ca605bcd29a5a37697ca66e533ae817ced71b6bf275c...,2,0,1.0,2,...,"[40346, 28654, 49539, 169432, 214225, 116215, ...","[1, 3, 2, 9, 6, 11, 7, 5, 4, 8, 10]",4-2-3-1,2018-08-21 18:15:00,Leeds United,"{""Kalvin Phillips"": ""DMC"", ""Jamie Shackleton"":...","[155405, 221610, 98760, 57913, 220037, 38588, ...","[4, 2, 9, 3, 1, 5, 7, 8, 10, 11, 6]",4-1-4-1,2018-08-21 18:15:00
4,174dba7291174b4dbbfa9ea12dd944bb45bdd8ed905524...,990997,2018-08-22 18:45:00,126905d14981e6b97912ad4fec354035ccef26cb8ec4e1...,7,419088133137a53bfdb1b7e2e682d223d33a6fa075bbfe...,94,0,1.0,2,...,"[12150, 59115, 122806, 54764, 49773, 37339, 43...","[4, 6, 8, 9, 11, 7, 5, 2, 3, 1, 10]",4-4-1-1,2018-08-22 18:15:00,Brentford,"{""Daniel Bentley"": ""GK"", ""Ezri Konsa"": ""DCR"", ...","[79602, 199798, 115382, 114275, 176442, 223911...","[1, 5, 9, 8, 3, 6, 7, 10, 4, 11, 2]",4-2-3-1,2018-08-22 18:15:00
5,31f82721a217969c32d6239ef4aeaf4a9203beaa5ac9d2...,991002,2018-08-22 18:45:00,f6a6116c751e3a2176cd2b55439ea27ea3991b6025d68a...,19,ec66a4ab5c85262aaf1455ac9b64b610c5b72f5a7ab747...,103,1,0.0,2,...,"[59013, 18953, 435858, 169525, 235936, 125404,...","[4, 9, 7, 6, 3, 11, 8, 2, 10, 1, 5]",4-2-3-1,2018-08-22 18:15:00,Millwall,"{""Jake Cooper"": ""DCL"", ""Jordan Archer"": ""GK"", ...","[167895, 107084, 105711, 94217, 56984, 80199, ...","[6, 1, 7, 4, 5, 10, 2, 3, 8, 9, 11]",4-4-2,2018-08-22 18:15:00
6,219ef70c0e8a803ec1efdb793443edfaa32398690c7829...,991003,2018-08-22 18:45:00,aeb2f56fcedbcf4cd5c780179766996c7bf0b308064541...,5,f8daf96ad35eebf1c0a5886c72734ba7dec366d6637052...,108,0,1.0,2,...,"[156685, 15398, 154561, 165183, 61602, 167802,...","[11, 9, 1, 3, 4, 5, 6, 2, 10, 8, 7]",4-2-3-1,2018-08-22 18:15:00,Reading,"{""Leandro Bacuna"": ""DMR"", ""Andy Yiadom"": ""DR"",...","[74297, 87408, 154133, 86857, 17506, 112133, 2...","[4, 2, 8, 6, 5, 3, 1, 7, 10, 11, 9]",4-4-2,2018-08-22 18:15:00
7,509cf73000c78b994b989f096b620fa7245c4aa67bfa88...,991005,2018-08-22 18:45:00,9ee012a80cade2df55b71580bf5e238bcd6be6f696fdc1...,45,004df4470c810ec01b28dcedb34be62

## Popular betting markets

There are many types of bets available in football today but betting on the match outcome, known as the `1x2` market, is the most popular one by far. The punter can bet the 3 possible match outcomes: *Home-win*, *draw* or *away-win*. 

A second popular market, in particular in Asia, is known as Asian handicap and is directly linked to the game goal difference. It offers only 2 betting options known as *home* (1) or *away* (2) and assumes that one of the team will start with a handicap (known as the line) and the other with an advantage. There are 3 types of Asian handicap markets:

  - *Integer Lines*: if the handicap chosen is equal to the score difference, it can result in a bet being returned (fold or push).
  - *Half Lines*: the outcome of the wager is either a win or a loss.
  - *Quarter Lines*: Divide your wager equally between the two AH lines above and beyond. For instance, a handicap of -0.75 is really two handicaps for the price of one, with half of your bet being made at -0.5 and the other half at -1, both of which are given the same odds. In this case, we have a win, half-win, half-loss, or a loss.
  
Another popular market is known as *total goals* where the punter can bet on the total number of goals in a game being over or under a given number (known as the line). Similar to the Asian handicap market, 3 type of lines are available and the punter can end up with a win, half-win, money returned, half stake loss or total loss.

## PnL computation

### 1X2 market

The PnL of the 1x2 market depends on the match outcome (home-win/draw/away-win) and is *binary*: the punter can either make a profit and get his stake returned or loose his stake.

$$ profit_{1X2} =  Stake *(Odds \times -1)$$
With:

+ Stake : Invested money
+ Odds : 1X2 European/decimal betting odds (> 1)

In [ ]:
#| export


def pnl_1X2(
    selection: np.ndarray,  # The amount invested on each selection.
    outcome: np.ndarray,  # Game result (Binary side outcome), shape=(1,5).
    odds: np.ndarray,  # odds for the current game, shape=(1,5)
) -> np.ndarray:  # 1X2 PnL
    "Returns the 1X2 PnL."
    assert selection.shape == odds.shape, "odds and selection should be same shape!"
    assert outcome.shape == odds.shape, "odds and outcome should be same shape!"
    n = selection.shape[0]

    pnl = selection[:, :3] * outcome[:, :3] * odds[:, :3] - selection[:, :3]
    return pnl.sum(axis=1).reshape((n, -1))

In [ ]:
# Bet action (here, our action is betting 20% of our balance on away team).
selection = (
    np.array(
        [
            [0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.7, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
        ]
    )
    * 100.0
)

info = fixture.head(3)

# Game odds.
odds = info[
    [
        "preGameOdds1",
        "preGameOddsX",
        "preGameOdds2",
        "preGameAhHome",
        "preGameAhAway",
        "preGameOver",
        "preGameUnder",
    ]
].values

# Game result index(homewin -> 0 , draw -> 1, awaywin -> 2).
game_result = info["tgt_outcome"].values.astype(int)
binary_result = np.zeros_like(selection)
np.put_along_axis(binary_result, np.expand_dims(game_result, axis=1), 1, axis=1)

pnl_1X2(selection, binary_result, odds)

array([[-20.],
       [-70.],
       [-80.]])

### Asian markets

The PnL for the Asian market is more complicated. It is related to the game goal-difference/ total goals and the actual handicap line. It can result, as we described above, in 5 outcomes: win/half-win/fold/half-loss/loss.

$$ profit_{AH} = \begin{cases} 
     0.0 & \text{If outcome + line = 0} \\\\ 
     \frac{odds -1}{2}& \text{If outcome + line = 0.25}\\\\
     odds -1 & \text{If outcome + line >= 0.5} \\\\
     -0.5 & \text{If outcome + line = -0.25} \\\\
     -1.0 & \text{If outcome + line <= -0.5} \end{cases} $$
 
 With:

+ outcome : game goal difference or total goals
+ line: Asian handicap line
+ Odds : Asian odds

In [ ]:
#| export


def _asian_unit_pnl(advantage, asian_odds):
    "provides the asian outcome given for a unit bet."

    if advantage == 0:
        return 0.0
    elif advantage == 0.25:
        return (asian_odds - 1) * 0.5
    elif advantage == -0.25:
        return -0.5
    elif advantage >= 0.5:
        return asian_odds - 1
    elif advantage <= -0.5:
        return -1.0

#### Asian handicap market

In [ ]:
#| export


def pnl_ah(
    selection: np.ndarray,  # The amount invested on each selection; shape n x 7; 4th and 5th index are asian handicap
    odds: np.ndarray,  # market odds in 1|X|2|A1|A2|O|U order; shape n x 7
    obs_gd: np.ndarray,  # Game goal-difference; shape (n,)
    ah_line: np.ndarray,  # Asian line could be integer, half or quarter line; shape (n,)
) -> np.ndarray:  # Asian Handicap PnL.
    "Returns the Asian Handicap PnL"

    # check dimension
    n = selection.shape[0]
    obs_gd = obs_gd.reshape((n, -1))
    ah_line = ah_line.reshape((n, -1))
    assert selection.shape == odds.shape, "odds and selection should be same shape!"

    ah_selection = selection[:, 3:5]
    ah_odds = odds[:, 3:5]

    # change line sign if betting on away
    ah_idx = np.where(ah_selection > 0)
    flip_sign = np.zeros_like(ah_line)
    flip_sign[ah_idx[0], 0] = np.where(ah_idx[1] == 0, 1, -1)

    # reshape odds
    _ah_odds = np.zeros_like(ah_line)
    _ah_odds[ah_idx[0], 0] = ah_odds[ah_idx]

    # reshape selection
    _ah_sel = np.zeros_like(ah_line)
    _ah_sel[ah_idx[0], 0] = ah_selection[ah_idx]

    _pnl_ah_v = np.vectorize(_asian_unit_pnl)
    _unit_pnl = _pnl_ah_v(
        obs_gd * flip_sign + ah_line * flip_sign, _ah_odds.reshape((n, -1))
    )

    return _unit_pnl * _ah_sel.reshape((n, -1))

In [ ]:
selection = (
    np.array(
        [
            [0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            [0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0],
        ]
    )
    * 100.0
)

info = fixture.head(3)

# Game odds.
odds = info[
    [
        "preGameOdds1",
        "preGameOddsX",
        "preGameOdds2",
        "preGameAhHome",
        "preGameAhAway",
        "preGameOver",
        "preGameUnder",
    ]
].values

obs_gd = info.tgt_gd.values
ah_line = info.preGameAhLineId.values


pnl_ah(selection, odds, obs_gd, ah_line)

array([[-50.],
       [  0.],
       [-20.]])

#### Total goals market

In [ ]:
#| export

def pnl_total(
    selection: np.ndarray,  # The amount invested on each selection; shape n x 7; last 2 are for over/under
    odds: np.ndarray,  # market odds in 1|X|2|A1|A2|O|U order; shape n x 7
    obs_total: np.ndarray,  # Game total goals; shape (n,)
    total_line: np.ndarray,  # Asian total goal line could be integer, half or quarter line; shape (n,)
) -> np.ndarray:  # Asian Total goal PnL.
    "Returns the Asian total goal PnL"

    # check dimension
    n = selection.shape[0]
    obs_total = obs_total.reshape((n, -1))
    total_line = total_line.reshape((n, -1))
    assert selection.shape == odds.shape, "odds and selection should be same shape!"

    ah_selection = selection[:, -2:]
    ah_odds = odds[:, -2:]

    # change line sign if betting on away
    ah_idx = np.where(ah_selection > 0)
    flip_sign = np.zeros_like(total_line)
    flip_sign[ah_idx[0], 0] = np.where(ah_idx[1] == 0, 1, -1)

    # reshape odds
    _ah_odds = np.zeros_like(total_line)
    _ah_odds[ah_idx[0], 0] = ah_odds[ah_idx]

    # reshape selection
    _ah_sel = np.zeros_like(total_line)
    _ah_sel[ah_idx[0], 0] = ah_selection[ah_idx]

    _pnl_ah_v = np.vectorize(_asian_unit_pnl)
    _unit_pnl = _pnl_ah_v(
        obs_total * flip_sign - total_line * flip_sign, _ah_odds.reshape((n, -1))
    )

    return _unit_pnl * _ah_sel.reshape((n, -1))

In [ ]:
selection = (
    np.array(
        [
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0],
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.65],
        ]
    )
    * 100.0
)

info = fixture.head(3)

# Game odds.
odds = info[
    [
        "preGameOdds1",
        "preGameOddsX",
        "preGameOdds2",
        "preGameAhHome",
        "preGameAhAway",
        "preGameOver",
        "preGameUnder",
    ]
].values

obs_total = info.tgt_homeTeamGoals.values + info.tgt_awayTeamGoals.values
total_line = info.preGameTotalLineId.values


pnl_total(selection, odds, obs_total, total_line)

array([[ 22.75],
       [ 27.9 ],
       [-65.  ]])

## Pnl wrapper

This function computes the total PnL:

In [ ]:
#| export


def pnl(
    selection: np.ndarray,  # The amount invested on each selection; shape n x 7; 
    odds: np.ndarray,  # market odds in 1|X|2|A1|A2|O|U order; shape n x 7
    obs_gd: np.ndarray,  # Game goal-difference; shape (n,)
    obs_total:np.ndarray,  # Game total-goals; shape (n,)
    ah_line: np.ndarray,  # Asian line could be integer, half or quarter line; shape (n,)
    total_line: np.ndarray,  # Asian total line could be integer, half or quarter line; shape (n,)
) -> np.ndarray:  # total PnL.
    "Returns the total PnL"
    # check dimesnion
    n = selection.shape[0]
    assert selection.shape == odds.shape, "odds and selection should be same shape!"

    if len(obs_gd.shape) > 1 and obs_gd.shape[1] == 1:
        obs_gd = obs_gd.squeeze(1)
    
    if len(obs_total.shape) > 1 and obs_total.shape[1] == 1:
        obs_total = obs_total.squeeze(1)
        
    # 1x2 results
    binary_result = np.zeros_like(selection)
    binary_result[obs_gd > 0, 0] = 1
    binary_result[obs_gd == 0, 1] = 1
    binary_result[obs_gd < 0, 2] = 1

    # 1x2 pnl
    _pnl_1x2 = pnl_1X2(selection, binary_result, odds)

    # ah-line
    _pnl_ah = pnl_ah(selection, odds, obs_gd, ah_line)
    
    # total
    _pnl_total = pnl_total(selection, odds, obs_total, total_line)

    _total_pnl = _pnl_1x2 + _pnl_ah + _pnl_total

    return _total_pnl

In [ ]:
selection = (
    np.array(
        [
            [0, 0.0, 0, 0.0, 0.3, 0.0, 0.0],  # bet on asian 2
            [0.3, 0, 0, 0, 0, 0.0, 0.0],  # bet on home
            [0, 0, 0.6, 0.0, 0, 0.0, 0.0],  # bet on away
            [0, 0, 0.0, 0.6, 0, 0.0, 0.0],  # bet on asian 1
            [0, 0.5, 0.0, 0.0, 0, 0.0, 0.0],  # bet on X
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5], # bet on Under
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.3, 0.0], # bet on Over
            [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], # not bet
        ]
    )
    * 100.0
)

info = fixture.head(selection.shape[0])

# Game odds.
odds = info[
    [
        "preGameOdds1",
        "preGameOddsX",
        "preGameOdds2",
        "preGameAhHome",
        "preGameAhAway",
        "preGameOver",
        "preGameUnder",
    ]
].values

obs_gd = info.tgt_gd.values
ah_line = info.preGameAhLineId.values
obs_total = info.tgt_homeTeamGoals.values + info.tgt_awayTeamGoals.values
total_line = info.preGameTotalLineId.values

pnl(selection=selection,
    odds=odds,
    obs_gd=obs_gd,
    ah_line=ah_line,
    obs_total=obs_total,
    total_line=total_line)

array([[-30. ],
       [-30. ],
       [ 85.2],
       [  0. ],
       [126.5],
       [-50. ],
       [ 29.1],
       [  0. ]])

In [ ]:
#| hide

import nbdev

nbdev.nbdev_export()